In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import numpy as np
import utils
import readin_v5
import os
import time

In [2]:
class yololike(nn.Module):
    
    def __init__(self):
        super(yololike,self).__init__()
    
        # layer1, 3*360*640  --> 16*360*640 
        self.conv1=nn.Conv2d(3,16,kernel_size=3,padding=1)
        # maxpool,16*360*640 --> 16*180*320
        self.pool1=nn.MaxPool2d(2,2)
        self.norm1=nn.BatchNorm2d(16)
    
        # layer2, 16*180*320 --> 32*180*320 
        self.conv2=nn.Conv2d(16,32,kernel_size=3,padding=1)
        # maxpool,32*180*320 --> 32*90*160
        self.pool2=nn.MaxPool2d(2,2)
        self.norm2=nn.BatchNorm2d(32)
        
        # layer3, 32*90*160 --> 64*90*160 
        self.conv3=nn.Conv2d(32,64,kernel_size=3,padding=1)
        # maxpool,64*90*160 --> 64*45*80
        self.pool3=nn.MaxPool2d(2,2)
        self.norm3=nn.BatchNorm2d(64)
        
        # layer4, 64*45*80 --> 128*45*80
        self.conv4=nn.Conv2d(64,128,kernel_size=3,padding=1)
        # maxpool,128*45*80 --> 128*23*40
        self.pool4=nn.MaxPool2d(2,2,padding=[1,0])
        self.norm4=nn.BatchNorm2d(128)
        
        # layer5, 128*23*40 --> 256*23*40
        self.conv5=nn.Conv2d(128,256,kernel_size=3,padding=1)
        # maxpool,256*23*40 --> 256*12*20
        self.pool5=nn.MaxPool2d(2,2,padding=[1,0])
        self.norm5=nn.BatchNorm2d(256)
        
        # layer6, 256*12*20 --> 512*12*20 
        self.conv6=nn.Conv2d(256,512,kernel_size=3,padding=1)
        # maxpool,512*12*20 --> 512*6*10?
        self.pool6=nn.MaxPool2d(2,2)
        self.norm6=nn.BatchNorm2d(512)
        
        # layer7, 512* --> 1024*
        self.conv7=nn.Conv2d(512,1024,kernel_size=3,padding=1)
        self.norm7=nn.BatchNorm2d(1024)
        
        ##################################
        # Location Network, the same as DeepZ
        ##################################
        
        # layer8, 1024* --> 512* 
        self.conv8=nn.Conv2d(1024,512,kernel_size=3,padding=1)
        self.norm8=nn.BatchNorm2d(512)
        
        # layer9, 512* --> 125*6*10=7500
        self.conv9=nn.Conv2d(512,125,kernel_size=1,padding=0)
        
        # layer10-a, region
        self.line10=nn.Linear(7500,84)
        
    def forward(self,x):
        
        # Layer1,activation=leaky
        x=self.conv1(x)
        x=F.leaky_relu(x)
        x=self.pool1(x)
        x=self.norm1(x)
        
        # layer2,activation=leaky
        x=self.conv2(x)
        x=F.leaky_relu(x)
        x=self.pool2(x)
        x=self.norm2(x)
        
        # layer3,activation=leaky
        x=self.conv3(x)
        x=F.leaky_relu(x)
        x=self.pool3(x)
        x=self.norm3(x)
        
        # layer4,activation=leaky
        x=self.conv4(x)
        x=F.leaky_relu(x)
        x=self.pool4(x)
        x=self.norm4(x)
        
        # layer5,activation=leaky
        x=self.conv5(x)
        x=F.leaky_relu(x)
        x=self.pool5(x)
        x=self.norm5(x)
        #print(x.size())
        
        # layer6,activation=leaky
        x=self.conv6(x)
        x=F.leaky_relu(x)
        x=self.pool6(x)
        x=self.norm6(x)
        #print(x.size())
        # layer7,activation=leaky
        x=self.conv7(x)
        x=F.leaky_relu(x)
        x=self.norm7(x)
        #print(x.size())
        # layer8,activation=leaky
        x=self.conv8(x)
        x=F.leaky_relu(x)
        x=self.norm8(x)
        #print(x.size())
        # layer9,activation=leaky
        x=self.conv9(x)
        x=F.leaky_relu(x)
        #print(x.size())
        
        # layer10,activation=leaky
        x=x.view(-1,7500)
        x=self.line10(x)
        
        return(x)

In [3]:
def cal_iou(infer,target):
    x=infer[0]
    y=infer[1]
    w=infer[2]
    h=infer[3]
    xt=target[0]
    yt=target[1]
    wt=target[2]
    ht=target[3]
    # case 0: when contained by the target
    if ((x+w)>=xt) and ((y+h)>=yt) and ((x+w)<=xt+wt) and ((y+h)<=yt+ht) and (x>=xt) and (y>=yt) and (x<=xt+wt) and (y<=yt+ht):
        same=w*h
        iou=same/(wt*ht)
    # case 1: when right upper part is the same
    elif ((x+w)>=xt) and ((y+h)>=yt) and ((x+w)<=xt+wt) and ((y+h)<=yt+ht):
        same=(x+w-xt)*(y+h-yt)
        iou=same/(w*h+wt*ht-same)
    # case 2: right down part
    elif ((x+w)>=xt) and (y>=yt) and ((x+w)<=xt+wt) and (y<=yt+ht):
        same=(x+w-xt)*(yt+ht-y)
        iou=same/(w*h+wt*ht-same)
    # case 3: left upper part
    elif (x>=xt) and ((y+h)>=yt) and (x<=xt+wt) and ((y+h)<=yt+ht):
        same=(xt+wt-x)*(y+h-yt)
        iou=same/(w*h+wt*ht-same)
    # case 4: left down part
    elif (x>=xt) and (y>=yt) and (x<=xt+wt) and (y<=yt+ht):
        same=(xt+wt-x)*(yt+ht-y)
        iou=same/(w*h+wt*ht-same)
    # case 5: contain the target
    elif ((xt+wt)>=x) and ((yt+ht)>=y) and ((xt+wt)<=x+w) and ((yt+ht)<=y+h) and (xt>=x) and (yt>=y) and (xt<=x+w) and (yt<=y+h):
        same=wt*ht
        iou=same/(w*h)
    else :
        same=0.0
        iou=0.0
    del x,y,w,h,xt,yt,wt,ht,same
    return iou

In [4]:
def get_iou(out_prob,real_label,bs):
    iou_count=0.0
    for i in range(bs):
        iou_count=iou_count+cal_iou(out_prob[i],real_label[i])
    return(iou_count/bs)

In [5]:
def get_accu(out_prob,real_label,bs):
    count=0
    for i in range(bs):
        temp=0.0
        maxp=0
        for j in range(80):
            if temp<out_prob[i,j]:
                temp=out_prob[i,j]
                maxp=j
        if maxp==real_label[i,0]:
            count=count+1
    return(count)

In [25]:
mydir=os.getcwd()
train_data_dir=mydir+"\\train_data"
train_label_dir=mydir+"\\train_label"
small_data_dir=mydir+"\\small_data"
small_label_dir=mydir+"\\small_label"

net = yololike()
origname="yolo2tiny_v2_5-50_6000"
record=open(origname+"_record.txt",mode='a')

bs=20
lr=0.1
epoch_num=2
train_data_num=100
train_total=6000
small_num=900
iter_num=int(train_data_num/bs)
criterion1 =nn.CrossEntropyLoss()
criterion2 =nn.MSELoss()

start_time = time.time()
for epoch in range(epoch_num):
    print("lr=",lr)
    for step in range(int(train_total/train_data_num)):
        if step>0 and step%5==0 and lr>0.0002:
            del optimizer
            lr=lr/1.4
            print("lr=",lr)
        optimizer=torch.optim.SGD(net.parameters(),lr)
        
        # read in the data
        train_data = readin_v5.read_part_image(train_data_dir,step*100,100)
        train_label= readin_v5.read_part_xml(train_label_dir,step*100,100)
        print(train_data.size(),train_data.type())
        #train_label=train_label[:,0]
        
        accu_count=0
        step_loss=0.0
        iou_count=0.0
        for iter in range(iter_num):
            if iter>0 :
                del indices
                del minibatch_data
                del minibatch_label
                del inputs
                del prob
                #del log_prob
                del loss
                
            # create a minibatch
            indices=iter*bs
            minibatch_data = train_data[indices:indices+bs]
            minibatch_label= train_label[indices:indices+bs]
            #print(minibatch_label.type())
    
            # feed the input to the net
            inputs=minibatch_data
            inputs.requires_grad_()
            prob=net(inputs)
    
            # update the weights
            #log_prob=torch.log(prob)
            #loss=criterion(log_prob, minibatch_label)
            loss=criterion1(prob[:,0:80], minibatch_label[:,0])+4*criterion2(prob[:,80:84]/10, minibatch_label[:,1:5].float()/100)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            step_loss=step_loss+loss.item()
            
            # get the accuracy
            accu_batch=get_accu(prob,minibatch_label,bs)
            accu_count=accu_count+accu_batch
            iou_batch=get_iou(prob[:,80:84]/10,minibatch_label[:,1:5].float()/100,bs)
            iou_count=iou_count+iou_batch
            print("batch no.",step*iter_num+iter," loss:",'{0:.2f}'.format(loss.item()),
                  "time:",'{0:.2f}'.format(time.time()-start_time)," accu=",accu_batch," iou=",'{0:.4f}'.format(iou_batch))
        print("step loss: ",'{0:.2f}'.format(step_loss/iter_num),"accu=",'{0:.2f}'.format(accu_count/train_data_num),
              " iou="+'{0:.4f}'.format(iou_count/iter_num))
    del train_data
    del train_label
    del indices
    del minibatch_data
    del minibatch_label
    del inputs
    del prob
    del loss
    
    # save the net data
    net_name=origname+"_epoch_"+str(epoch)+".pt"
    torch.save(net.state_dict(),net_name)
    
    small_loss=0.0
    small_accu=0
    small_iou=0.0
    for step in range(int(small_num/100)):
    
        # read in the data
        small_data = readin_v5.read_part_image(small_data_dir,step*100,100)
        small_label= readin_v5.read_part_xml(small_label_dir,step*100,100)
        print("test:",small_data.size(),small_data.type())
        
        accu_count=0
        step_loss=0.0
        iou_count=0.0
        for iter in range(iter_num):
            if iter>0 :
                del indices
                del minibatch_data
                del minibatch_label
                del prob
                del loss
                del prob_xy
                del real_xy
                
            # create a minibatch
            indices=iter*bs
            minibatch_data = small_data[indices:indices+bs]
            minibatch_label= small_label[indices:indices+bs]
            #print(minibatch_label.type())
    
            # feed the input to the net
            prob=net(minibatch_data)
            loss=criterion1(prob[:,0:80], minibatch_label[:,0])+4*criterion2(prob[:,80:84]/10, minibatch_label[:,1:5].float()/100)
            step_loss=step_loss+loss.item()
            
            # get the accuracy
            accu_batch=get_accu(prob,minibatch_label,bs)
            accu_count=accu_count+accu_batch
            prob_xy=prob[:,80:84]/10
            real_xy=minibatch_label[:,1:5].float()/100
            iou_batch=get_iou(prob_xy,real_xy,bs)
            iou_item=iou_batch.item()
            del iou_batch
            iou_count=iou_count+iou_item
            print("small_batch no.",step*iter_num+iter," loss:",'{0:.2f}'.format(loss.item()),
                  "time:",'{0:.2f}'.format(time.time()-start_time)," accu=",accu_batch," iou=",'{0:.4f}'.format(iou_item))
        small_loss=small_loss+step_loss
        small_accu=small_accu+accu_count
        small_iou=small_iou+iou_count
        print("small_step loss: ",'{0:.2f}'.format(step_loss/iter_num),"accu=",'{0:.2f}'.format(accu_count/train_data_num),
              " iou="+'{0:.4f}'.format(iou_count/iter_num))
    del small_data
    del small_label
    del indices
    del minibatch_data
    del minibatch_label
    del prob
    del loss
    del prob_xy
    del real_xy
    print("small loss:",'{0:.2f}'.format(small_loss/(small_num/bs)),"accu=",'{0:.2f}'.format(small_accu/small_num),
          " iou="+'{0:.4f}'.format(small_iou/(small_num/bs)))
    

lr= 0.1
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 0  loss: 16.61 time: 7.91  accu= 1  iou= 0.0000
batch no. 1  loss: 10.40 time: 14.17  accu= 0  iou= 0.0000
batch no. 2  loss: 5.91 time: 20.36  accu= 0  iou= 0.0986
batch no. 3  loss: 11.39 time: 26.51  accu= 1  iou= 0.0120
batch no. 4  loss: 9.61 time: 32.57  accu= 1  iou= 0.0071
step loss:  10.78 accu= 0.03  iou=0.0235
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 5  loss: 13.17 time: 39.32  accu= 0  iou= 0.0594
batch no. 6  loss: 11.11 time: 45.43  accu= 2  iou= 0.0503
batch no. 7  loss: 11.73 time: 51.54  accu= 1  iou= 0.0412
batch no. 8  loss: 14.48 time: 57.59  accu= 0  iou= 0.0000
batch no. 9  loss: 38.72 time: 63.77  accu= 0  iou= 0.0000
step loss:  17.84 accu= 0.03  iou=0.0302
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 10  loss: 36.96 time: 70.74  accu= 2  iou= 0.0000
batch no. 11  loss: 43.39 time: 76.66  accu= 0  iou= 0.0000
batch no. 12  loss: 55.81 time: 82.78  accu= 0  iou= 0.0

torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 105  loss: 3.63 time: 658.71  accu= 5  iou= 0.0635
batch no. 106  loss: 3.75 time: 664.77  accu= 8  iou= 0.0907
batch no. 107  loss: 3.50 time: 670.78  accu= 8  iou= 0.0628
batch no. 108  loss: 3.59 time: 676.76  accu= 6  iou= 0.0631
batch no. 109  loss: 3.77 time: 682.88  accu= 9  iou= 0.0566
step loss:  3.65 accu= 0.36  iou=0.0673
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 110  loss: 3.09 time: 689.62  accu= 9  iou= 0.1017
batch no. 111  loss: 3.61 time: 695.52  accu= 5  iou= 0.0902
batch no. 112  loss: 3.05 time: 701.57  accu= 6  iou= 0.0730
batch no. 113  loss: 3.54 time: 707.65  accu= 6  iou= 0.0952
batch no. 114  loss: 3.60 time: 713.77  accu= 7  iou= 0.0124
step loss:  3.38 accu= 0.33  iou=0.0745
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 115  loss: 4.13 time: 720.79  accu= 3  iou= 0.0902
batch no. 116  loss: 3.93 time: 726.92  accu= 4  iou= 0.0909
batch no. 117  loss: 3.41 time: 732.85  a

batch no. 207  loss: 3.06 time: 1288.79  accu= 10  iou= 0.1723
batch no. 208  loss: 2.74 time: 1294.77  accu= 12  iou= 0.0633
batch no. 209  loss: 3.30 time: 1300.57  accu= 10  iou= 0.0356
step loss:  2.85 accu= 0.55  iou=0.1096
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 210  loss: 2.39 time: 1307.20  accu= 8  iou= 0.0349
batch no. 211  loss: 2.19 time: 1313.04  accu= 12  iou= 0.0747
batch no. 212  loss: 2.76 time: 1319.03  accu= 10  iou= 0.0493
batch no. 213  loss: 2.52 time: 1324.96  accu= 11  iou= 0.0822
batch no. 214  loss: 2.95 time: 1330.87  accu= 11  iou= 0.0765
step loss:  2.56 accu= 0.52  iou=0.0635
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 215  loss: 2.47 time: 1337.62  accu= 10  iou= 0.0632
batch no. 216  loss: 2.83 time: 1343.83  accu= 7  iou= 0.0295
batch no. 217  loss: 3.03 time: 1349.94  accu= 9  iou= 0.0819
batch no. 218  loss: 2.69 time: 1355.99  accu= 12  iou= 0.1319
batch no. 219  loss: 2.39 time: 1362.19  accu= 11  iou= 0.0962
step l

small_batch no. 8  loss: 2.74 time: 1879.53  accu= 13  iou= 0.0685
small_batch no. 9  loss: 3.64 time: 1882.15  accu= 9  iou= 0.0559
small_step loss:  2.96 accu= 0.48  iou=0.1081
test: torch.Size([100, 3, 360, 640]) torch.FloatTensor
small_batch no. 10  loss: 2.09 time: 1885.40  accu= 13  iou= 0.1346
small_batch no. 11  loss: 2.72 time: 1887.94  accu= 7  iou= 0.0646
small_batch no. 12  loss: 2.25 time: 1890.59  accu= 13  iou= 0.0522
small_batch no. 13  loss: 2.55 time: 1893.29  accu= 15  iou= 0.1217
small_batch no. 14  loss: 3.29 time: 1895.93  accu= 5  iou= 0.1196
small_step loss:  2.58 accu= 0.53  iou=0.0985
test: torch.Size([100, 3, 360, 640]) torch.FloatTensor
small_batch no. 15  loss: 2.45 time: 1899.12  accu= 14  iou= 0.0803
small_batch no. 16  loss: 3.16 time: 1901.85  accu= 7  iou= 0.0667
small_batch no. 17  loss: 2.08 time: 1904.42  accu= 15  iou= 0.1000
small_batch no. 18  loss: 2.41 time: 1907.09  accu= 14  iou= 0.0499
small_batch no. 19  loss: 2.64 time: 1909.76  accu= 9  i

batch no. 61  loss: 3.07 time: 2360.38  accu= 11  iou= 0.0449
batch no. 62  loss: 2.83 time: 2366.42  accu= 8  iou= 0.0337
batch no. 63  loss: 2.00 time: 2372.34  accu= 12  iou= 0.0891
batch no. 64  loss: 3.28 time: 2378.44  accu= 6  iou= 0.0378
step loss:  2.63 accu= 0.50  iou=0.0571
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 65  loss: 3.04 time: 2385.57  accu= 10  iou= 0.1016
batch no. 66  loss: 2.38 time: 2391.51  accu= 8  iou= 0.1129
batch no. 67  loss: 2.70 time: 2397.58  accu= 14  iou= 0.0265
batch no. 68  loss: 2.11 time: 2403.69  accu= 11  iou= 0.1147
batch no. 69  loss: 2.32 time: 2409.74  accu= 15  iou= 0.0805
step loss:  2.51 accu= 0.58  iou=0.0873
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 70  loss: 3.31 time: 2416.40  accu= 10  iou= 0.0576
batch no. 71  loss: 2.45 time: 2422.42  accu= 10  iou= 0.1549
batch no. 72  loss: 2.38 time: 2428.35  accu= 10  iou= 0.0928
batch no. 73  loss: 2.45 time: 2434.33  accu= 13  iou= 0.0886
batch no. 74  loss:

batch no. 163  loss: 2.69 time: 2986.57  accu= 11  iou= 0.1073
batch no. 164  loss: 2.44 time: 2992.75  accu= 9  iou= 0.0630
step loss:  2.21 accu= 0.66  iou=0.1200
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 165  loss: 2.18 time: 2999.54  accu= 12  iou= 0.0624
batch no. 166  loss: 1.87 time: 3005.65  accu= 12  iou= 0.0611
batch no. 167  loss: 2.12 time: 3011.71  accu= 13  iou= 0.1551
batch no. 168  loss: 1.83 time: 3017.58  accu= 14  iou= 0.1356
batch no. 169  loss: 2.71 time: 3023.70  accu= 10  iou= 0.0825
step loss:  2.14 accu= 0.61  iou=0.0993
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 170  loss: 2.44 time: 3030.61  accu= 10  iou= 0.1561
batch no. 171  loss: 3.66 time: 3036.63  accu= 5  iou= 0.0725
batch no. 172  loss: 3.54 time: 3042.72  accu= 7  iou= 0.0337
batch no. 173  loss: 2.25 time: 3048.75  accu= 11  iou= 0.0624
batch no. 174  loss: 2.00 time: 3054.67  accu= 11  iou= 0.1094
step loss:  2.78 accu= 0.44  iou=0.0868
lr= 0.00023425849983812192
to

torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 265  loss: 3.19 time: 3615.59  accu= 8  iou= 0.0802
batch no. 266  loss: 2.55 time: 3621.72  accu= 11  iou= 0.1497
batch no. 267  loss: 2.51 time: 3627.85  accu= 9  iou= 0.0927
batch no. 268  loss: 2.34 time: 3633.81  accu= 13  iou= 0.0449
batch no. 269  loss: 1.92 time: 3639.88  accu= 14  iou= 0.0653
step loss:  2.50 accu= 0.55  iou=0.0866
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 270  loss: 1.96 time: 3646.54  accu= 13  iou= 0.0552
batch no. 271  loss: 2.39 time: 3652.58  accu= 13  iou= 0.0633
batch no. 272  loss: 2.33 time: 3658.57  accu= 12  iou= 0.1116
batch no. 273  loss: 1.85 time: 3664.58  accu= 16  iou= 0.0639
batch no. 274  loss: 2.19 time: 3670.57  accu= 12  iou= 0.0658
step loss:  2.15 accu= 0.66  iou=0.0719
torch.Size([100, 3, 360, 640]) torch.FloatTensor
batch no. 275  loss: 1.95 time: 3677.34  accu= 13  iou= 0.0842
batch no. 276  loss: 2.17 time: 3683.25  accu= 13  iou= 0.1299
batch no. 277  loss

In [24]:
print(train_label[0:10])
print(4*criterion2(prob[:,80:84]/10, minibatch_label[:,1:5].float()/100))

tensor([[ 42, 332, 155,  32,  84],
        [ 50, 409,  71,  47, 113],
        [ 18, 309, 176,  23,  27],
        [ 32, 323, 142,   8,  35],
        [ 52, 291, 149,  24,  55],
        [ 14, 309, 206,  28,  33],
        [  0, 302, 249, 138,  70],
        [  4, 395, 204,  39,  49],
        [ 53, 299, 166,  36,  56],
        [ 11, 405, 248,  13,  22]])


NameError: name 'prob' is not defined

In [ ]:
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
print(target)
#output = loss(input, target)
xy=prob[:,80:84]/100
xytrue=minibatch_label[:,1:5].float()/100
print(xytrue)
print(xytrue.type(),xytrue.size())
output = loss(xy,xytrue)
print(output.size())


In [ ]:
def cal_iou(infer,target):
    x=infer[0]
    y=infer[1]
    w=infer[2]
    h=infer[3]
    xt=target[0]
    yt=target[1]
    wt=target[2]
    ht=target[3]
    # case 0: when contained by the target
    if ((x+w)>=xt) and ((y+h)>=yt) and ((x+w)<=xt+wt) and ((y+h)<=yt+ht) 
    and (x>=xt) and (y>=yt) and (x<=xt+wt) and (y<=yt+ht):
        same=w*h
        iou=same/(wt*ht)
    # case 1: when right upper part is the same
    elif ((x+w)>=xt) and ((y+h)>=yt) and ((x+w)<=xt+wt) and ((y+h)<=yt+ht):
        same=(x+w-xt)*(y+h-yt)
        iou=same/(w*h+wt*ht-same)
    # case 2: right down part
    elif ((x+w)>=xt) and (y>=yt) and ((x+w)<=xt+wt) and (y<=yt+ht):
        same=(x+w-xt)*(yt+ht-y)
        iou=same/(w*h+wt*ht-same)
    # case 3: left upper part
    elif (x>=xt) and ((y+h)>=yt) and (x<=xt+wt) and ((y+h)<=yt+ht):
        same=(xt+wt-x)*(y+h-yt)
        iou=same/(w*h+wt*ht-same)
    # case 4: left down part
    elif (x>=xt) and (y>=yt) and (x<=xt+wt) and (y<=yt+ht):
        same=(xt+wt-x)*(yt+ht-y)
        iou=same/(w*h+wt*ht-same)
    # case 5: contain the target
    elif ((xt+wt)>=x) and ((yt+ht)>=y) and ((xt+wt)<=x+w) and ((yt+ht)<=y+h) 
    and (xt>=x) and (yt>=y) and (xt<=x+w) and (yt<=y+h):
        same=wt*ht
        iou=same/(w*h)
    else :
        same=0.0
        iou=0.0
    del x,y,w,h,xt,yt,wt,ht,same
    return iou